In [ ]:
import requests
import json
import pandas as pd
from collections import defaultdict

client_id = 'client_230eaf0def104b549ba6fe40ef5a348b'
client_secret = 'secret_5ddff1be1b346fb4481a46cfb127c0f1'
auth_url = 'https://auth.domain.com.au/v1/connect/token'

property_metadata = defaultdict(dict)

def get_property(page, postcode):
    response = requests.post(auth_url, data = {
                        'client_id':client_id,
                        'client_secret':client_secret,
                        'grant_type':'client_credentials',
                        'scope':"api_agencies_read api_listings_read",
                        'Content-Type':'text/json'
                        })
    json_res = response.json()
    access_token=json_res['access_token']

    search_parameters = {
       "listingType": "Rent",
        "locations": [
            {
                "state": "VIC",   
                "postcode": postcode,
                "includeSurroundingSuburbs": False
            }
        ],
        "pageSize": 100,
        "pageNumber": page
    }

    auth = {'Authorization':'Bearer ' + access_token}
    url = "https://api.domain.com.au/v1/listings/residential/_search"
    request = requests.post(url, json=search_parameters, headers=auth)
    details = request.json()
    
    property_metadata[postcode] = details


for postcode in range(3000, 3999):
    for page in range(10):
        get_property(page, postcode)
    
    print(f'{postcode} done')

# output to example json in data/raw/
    with open("../data/raw/property.json", "w") as f: 
        json.dump(property_metadata, f)
